### Notebook 5: Optical Measurements Plotting
This notebook shows how one might go about plotting optical measurements data, including the ultraviolet reflectance, ultraviolet transmittance, near-infrared reflectance, and near-infrared transmittance. Later on, direct and indirect Tauc plots may be generated based on these measurements in addition to the thickness data collected from x-ray fluorescence. The band gaps are generated from analyses made along these indirect and direct Tauc plots.

In [1]:
import sys
import pandas as pd
sys.path.append('../lib')
#Note: When working in Windows environments, use:
#sys.path.append('..\lib')
from library import Library
from sample import Sample
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

After the proper modules have been imported, we use the Sample class to take a particular sample that has a complete spectrum of ultraviolet transmittance/reflectance and near-infrared transmittance/reflectance. Note that the near-infrared transmittance/reflectance spectra are much shorter than the ultraviolet spectra, so the DataFrame for all the optical spectra is padded with nulls. It is therefore necessary to remove these nulls using the pd.dropna() command.

In [2]:
optical_spectra = Sample(221101).spectra('optical')
uvit = sorted(zip(list(optical_spectra.uvit_wave),list(optical_spectra.uvit_response)))
uvir = sorted(zip(list(optical_spectra.uvir_wave),list(optical_spectra.uvir_response)))
nirt = sorted(zip(list(optical_spectra.nirt_wave.dropna()),list(optical_spectra.nirt_response.dropna())))
nirr = sorted(zip(list(optical_spectra.nirr_wave.dropna()),list(optical_spectra.nirr_response.dropna())))

TypeError: 'NoneType' object has no attribute '__getitem__'

The first step is to do a basic plotting of each of the transmittance/reflectance spectra. Note that the tails on each of the spectra tend to produce much noisier data. It may be necessary in some cases to smooth or eliminate these tails before proceeding.

In [ ]:
plt.figure(figsize = (10,6))
plt.title('UVIT Spectra for Sample 209139')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Transmittance')
plt.plot([i[0] for i in uvit],[i[1] for i in uvit])
plt.show()

plt.figure(figsize = (10,6))
plt.title('UVIR Spectra for Sample 209139')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.plot([i[0] for i in uvir],[i[1] for i in uvir])
plt.show()

plt.figure(figsize = (10,6))
plt.title('NIRT Spectra for Sample 209139')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Transmittance')
plt.plot([i[0] for i in nirt],[i[1] for i in nirt])
plt.show()

plt.figure(figsize = (10,6))
plt.title('NIRR Spectra for Sample 209139')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.plot([i[0] for i in nirr],[i[1] for i in nirr])
plt.show()

The code below takes care of all the manipulations necessary to generate Tauc plots. First the absorption coefficient is calculated, then the direct and indirect absorbances are calculated. The energy is a direct function of the wavelengh the measurement was taken at. The Tauc analysis compares the energy with the direct/indirect absorbances. The direct/indirect band gap is estimated about the first large "impulse", or sharp increase in direct/indirect absorbance. Using a rough linear approximation, one see how well the experimental Tauc plots line up with the band gap algorithm's approximation for the band gap.

In [ ]:
#Absorption coefficient
thickness = float(Sample(209139).properties().thickness)
transmittance = sorted(uvit+nirt)
reflectance = sorted(uvir+nirr)
absorption_coefficient = []
for i in range(0,len(transmittance)): #Equation for absorption coefficient
    absorption_coefficient.append(-1*np.log(abs(transmittance[i][1]/(1-reflectance[i][1])))/float(thickness*10**-4))
energy = [1240.0/i[0] for i in transmittance]

direct_absorbance = []
indirect_absorbance = []
for i in range(0,len(energy)):
    direct_absorbance.append((absorption_coefficient[i]**2)*(energy[i]**2))
    indirect_absorbance.append((abs(absorption_coefficient[i])**0.5)*(energy[i]**0.5))
normalized_direct_absorbance = [i/max(direct_absorbance) for i in direct_absorbance]
normalized_indirect_absorbance = [i/max(indirect_absorbance) for i in indirect_absorbance]
plt.figure()
plt.plot(energy,normalized_direct_absorbance)
plt.xlabel('Energy (eV)')
plt.ylabel('Normalized Direct Absorbance')
plt.title('Direct Tauc Plot')
plt.show()

plt.figure()
plt.plot(energy,normalized_indirect_absorbance)
plt.xlabel('Energy (eV)')
plt.ylabel('Normalized Direct Absorbance')
plt.title('Indirect Tauc Plot')
plt.show()

print('Direct Band Gap (eV): '+str(float(Sample(209139).properties().opt_direct_bandgap)))
print('Indirect Band Gap (eV): '+str(float(Sample(209139).properties().opt_indirect_bandgap)))

Using the same concepts found in the XRF notebook for plotting changes in concentration, one can generate a heatmap with respect to the direct or indirect band gap.

In [ ]:
opt_list = []
for i in Sample.search_by_ids(list(Library(6701).properties().sample_ids)[0]):
    opt_list.append(i.properties()[['position','opt_direct_bandgap','opt_indirect_bandgap']])

In [ ]:
opt_positions = [int(i.position) for i in opt_list]
opt_direct_bg = [float(i.opt_direct_bandgap) for i in opt_list]

In [ ]:
direct_bg_graph = sorted(zip(opt_positions,opt_direct_bg))
direct_bg_graph = [i[1] for i in direct_bg_graph]
direct_bg_graph_array = np.array(direct_bg_graph).reshape(4,11)
f, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(direct_bg_graph_array, annot=True, linewidths=.5, ax=ax,cmap = 'viridis')
plt.title('Direct Band Gap (eV) Across Library 6701')
plt.show()